In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from federated.Dataset import Dataset
from federated.Client import Client
from models.ClientModelStrategy import ClientModelStrategy
from knowledge_distillation import Logits, SoftTarget

## Dataset

In [2]:
logger = SummaryWriter()

In [3]:
data_path = "dataset\cinic-10"
synthetic_path = "diffusion\dataset\cifar_10"
dataset_id = "cinic10"
batch_size = 64
kd_batch_size = 128
num_clients = 2
dataset = Dataset(data_path, dataset_id, batch_size, kd_batch_size, num_clients, synthetic_path)
dataset.prepare_data("dirichlet")
synthetic_dataset = dataset.get_synthetic_data(None)

## Client

In [4]:
client_devices = [0] * (num_clients)
client_models = ClientModelStrategy.available["strategy_1"](num_clients)

In [18]:
params = {"num_classes": 10,
            "optimizer": optim.SGD, 
            "criterion": nn.CrossEntropyLoss, 
            "lr": 0.1 , 
            "momentum": 0.9, 
            "epochs": 10,
            "weight_decay": 1e-4, 
            "kd_optimizer": optim.SGD,
            "kd_criterion": SoftTarget,
            "kd_lr": 0.1,
            "kd_momentum": 0.9,
            "kd_alpha": 0.5,
            "kd_temperature": 1,
            "kd_epochs": 10,
            "eval_seed": 45, 
            "synthetic_epochs": 10}

In [6]:
client_id = 0
client = Client(client_id, 
                client_devices[client_id], 
                client_models[client_id](), 
                dataset.client_dataloaders[client_id],
                params,
                logger)

In [7]:
client.init_client()

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
# import resnet18
from torchvision.models import resnet18
from torchvision import transforms
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

demo_loader = DataLoader(CIFAR10(root="dataset", train=False, transform=transform, download=True), batch_size=64, shuffle=True)
model = resnet18(num_classes=10)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
epochs = 10
for epoch in range(epochs):
    correct = 0
    total = 0
    total_loss = 0
    for batch in demo_loader:
        data, target = batch
        # print(target[0])
        optimizer.zero_grad()
        output = model(data)
        # print(output[0])
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        total_loss += loss.item()
    print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, total_loss/len(demo_loader), correct/total))


## Server

In [7]:
from federated.Server import Server
from models.Models import Models

In [19]:
server_device = 0
server_model = Models.ResNet18()
server = Server(server_device, server_model, params, logger)

In [20]:
server.init_server()

In [21]:
server.synthetic_dataset = synthetic_dataset

In [23]:
server.synthetic_train()

In [24]:
server.params["synthetic_epochs"]

10